In [ ]:
print('boa tarde!')

In [1]:
%pip install pandas

  Using cached pytz-2024.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2024.2-py2.py3-none-any.whl.metadata (1.4 kB)
   ---------------------------------------- 0.0/11.5 MB ? eta -:--:--
   ---------- ----------------------------- 3.1/11.5 MB 20.6 MB/s eta 0:00:01
   ------------------------------ --------- 8.9/11.5 MB 25.2 MB/s eta 0:00:01
   ---------------------------------------- 11.5/11.5 MB 24.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/12.6 MB ? eta -:--:--
   ---------------------------------------  12.6/12.6 MB 60.7 MB/s eta 0:00:01
   ---------------------------------------- 12.6/12.6 MB 52.8 MB/s eta 0:00:00
Using cached pytz-2024.2-py2.py3-none-any.whl (508 kB)
Using cached tzdata-2024.2-py2.py3-none-any.whl (346 kB)
Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install numpy

Note: you may need to restart the kernel to use updated packages.


In [42]:
%pip install requests

Note: you may need to restart the kernel to use updated packages.


In [43]:
%pip install json

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement json (from versions: none)
ERROR: No matching distribution found for json


In [ ]:
%pip install psycopg2
%pip show psycopg2-binary

   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   ------------------------------------ --- 1.0/1.2 MB 12.7 MB/s eta 0:00:01
   ---------------------------------------- 1.2/1.2 MB 8.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [5]:
import psycopg2

In [29]:
import psycopg2

# Configurações de conexão
db_config = {
    "dbname": "Test",
    "user": "avnadmin",
    "password": "AVNS_9fZb3BkX9qGXxKpxsrZ",
    "host": "postgresql-iscac.f.aivencloud.com",  # ou o IP do servidor
    "port": 25674,  # Porta padrão do PostgreSQL
}

try:
    # Conectando ao banco de dados
    conn = psycopg2.connect(**db_config)
    cursor = conn.cursor()

    # SQL para criar a tabela, se não existir
    create_table_query = """
    CREATE TABLE IF NOT EXISTS tabela_test (
        Id SERIAL PRIMARY KEY,
        Name TEXT NOT NULL,
        Age TEXT NOT NULL,
        City TEXT
    );
    """
    cursor.execute(create_table_query)

    # Confirmar a criação da tabela
    conn.commit()
    print("Tabela criada (ou já existia).")

except Exception as e:
    print(f"Erro ao criar tabela: {e}")
    conn.rollback()

finally:
    if cursor:
        cursor.close()
    if conn:
        conn.close()


Tabela criada (ou já existia).


In [44]:
import psycopg2
import requests
import json
import pandas as pd

# URL da API
URL = 'https://opendata.emel.pt/cycling/bicyclecounter/locations'

# Fazer a requisição para a API
response_api = requests.get(URL)
print(response_api.status_code)

# Carregar os dados da API em um dicionário Python
data_loc = json.loads(response_api.text)

# Normalizar os dados JSON para um DataFrame
df_loc = pd.json_normalize(data_loc)

# Mostrar as informações do DataFrame
df_loc.info()

# Configurações de conexão com o banco de dados PostgreSQL
db_config = {
    "dbname": "Test",
    "user": "avnadmin",
    "password": "AVNS_9fZb3BkX9qGXxKpxsrZ",
    "host": "postgresql-iscac.f.aivencloud.com",  # ou o IP do servidor
    "port": 25674,  # Porta padrão do PostgreSQL
}

# Função para criar a tabela, caso ela não exista
def create_table(cursor):
    create_table_query = """
    CREATE TABLE IF NOT EXISTS bicycle_locations (
        id SERIAL PRIMARY KEY,
        location_id INTEGER NOT NULL,
        location_name TEXT NOT NULL,
        latitude FLOAT,
        longitude FLOAT,
        last_updated TIMESTAMP
    );
    """
    cursor.execute(create_table_query)

# Função para inserir ou atualizar os dados
def insert_or_update_data(cursor, df):
    for index, row in df.iterrows():
        # Verificar se já existe um dado com o mesmo location_id
        select_query = "SELECT id FROM bicycle_locations WHERE location_id = %s"
        cursor.execute(select_query, (row['locationId'],))
        existing_record = cursor.fetchone()

        # Preparar os valores
        location_id = row['locationId']
        location_name = row['locationName']
        latitude = row['latitude']
        longitude = row['longitude']
        last_updated = row['lastUpdated']

        # Se já existe, fazer um UPDATE
        if existing_record:
            update_query = """
            UPDATE bicycle_locations
            SET location_name = %s, latitude = %s, longitude = %s, last_updated = %s
            WHERE location_id = %s
            """
            cursor.execute(update_query, (location_name, latitude, longitude, last_updated, location_id))
        else:
            # Se não existe, fazer um INSERT
            insert_query = """
            INSERT INTO bicycle_locations (location_id, location_name, latitude, longitude, last_updated)
            VALUES (%s, %s, %s, %s, %s)
            """
            cursor.execute(insert_query, (location_id, location_name, latitude, longitude, last_updated))

# Função principal
def main():
    try:
        # Conectar ao banco de dados
        conn = psycopg2.connect(**db_config)
        cursor = conn.cursor()

        # Criar a tabela, caso não exista
        create_table(cursor)

        # Inserir ou atualizar os dados da API no banco de dados
        insert_or_update_data(cursor, df_loc)

        # Commit para salvar as alterações
        conn.commit()
        print("Dados inseridos ou atualizados com sucesso!")

    except Exception as e:
        print(f"Erro: {e}")
        conn.rollback()

    finally:
        if cursor:
            cursor.close()
        if conn:
            conn.close()

# Executar a função principal
if __name__ == "__main__":
    main()


403


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
import psycopg2
import csv

# Configurações de conexão com o banco de dados
db_config = {
    "dbname": "Test",
    "user": "avnadmin",
    "password": "AVNS_9fZb3BkX9qGXxKpxsrZ",
    "host": "postgresql-iscac.f.aivencloud.com",  # ou o IP do servidor
    "port": 25674,  # Porta padrão do PostgreSQL
}

# Caminho para o arquivo CSV
csv_file = "test_joana.csv"
table_name = "tabela_joana"  # Nome da tabela que será criada

def get_column_types(csv_file):
    """Lê as primeiras linhas do CSV e tenta inferir o tipo de dados das colunas."""
    with open(csv_file, 'r') as f:
        reader = csv.DictReader(f)
        first_row = next(reader)
        
    column_types = {}
    for col, value in first_row.items():
        # Tentar inferir os tipos de dados das colunas (int, float, ou text)
        try:
            float(value)  # Se for numérico, considerar como FLOAT
            column_types[col] = "FLOAT"
        except ValueError:
            column_types[col] = "TEXT"  # Caso contrário, é texto
    return column_types

def create_table(cursor, table_name, column_types):
    """Cria a tabela no PostgreSQL baseado nas colunas e seus tipos."""
    columns = f"datetime TEXT PRIMARY KEY, " + ", ".join([f"{col} {col_type}" for col, col_type in column_types.items() if col != "datetime"])
    create_table_query = f"CREATE TABLE IF NOT EXISTS {table_name} ({columns});"
    cursor.execute(create_table_query)

def insert_data_from_csv(cursor, csv_file, table_name, column_names):
    """Insere os dados do CSV na tabela."""
    with open(csv_file, 'r') as f:
        reader = csv.DictReader(f)
        for row in reader:
            values = [row[col] if row[col] != '' else None for col in column_names]
            insert_query = f"INSERT INTO {table_name} ({', '.join(column_names)}) VALUES ({', '.join(['%s'] * len(values))});"
            cursor.execute(insert_query, values)

def main():
    try:
        # Conectar ao banco de dados
        conn = psycopg2.connect(**db_config)
        cursor = conn.cursor()

        # Inferir os tipos de dados das colunas a partir do CSV
        column_types = get_column_types(csv_file)

        # Criar a tabela no banco de dados
        create_table(cursor, table_name, column_types)
        print(f"Tabela {table_name} criada com sucesso!")

        # Inserir os dados do CSV na tabela
        insert_data_from_csv(cursor, csv_file, table_name, column_types.keys())
        conn.commit()
        print("Dados inseridos com sucesso!")

    except Exception as e:
        print(f"Erro ao criar a tabela ou inserir os dados: {e}")
        conn.rollback()

    finally:
        if cursor:
            cursor.close()
        if conn:
            conn.close()

if __name__ == "__main__":
    main()


Tabela tabela_joana criada com sucesso!
Erro ao criar a tabela ou inserir os dados: duplicate key value violates unique constraint "tabela_joana_pkey"
DETAIL:  Key (datetime)=(2021-10-22T00:00:00) already exists.



In [45]:
from airflow import DAG
from airflow.operators.python_operator import PythonOperator
from datetime import datetime

# Definir a função Python que será chamada no workflow
def print_hello():
    print("Olá, este é um teste do Airflow!")

# Configurações padrão para o DAG
default_args = {
    'owner': 'airflow',
    'depends_on_past': False,
    'email_on_failure': False,
    'email_on_retry': False,
    'retries': 1,
}

# Definição do DAG
dag = DAG(
    'meu_workflow_print',  # Nome do DAG
    default_args=default_args,
    description='Um DAG simples para fazer um print',
    schedule_interval='@daily',  # Executa diariamente
    start_date=datetime(2025, 1, 1),
    catchup=False  # Evita execuções retroativas
)

# Tarefa usando PythonOperator
tarefa_print = PythonOperator(
    task_id='print_teste',  # Identificador único da tarefa
    python_callable=print_hello,  # Função Python a ser executada
    dag=dag
)

# Fluxo de execução
# Aqui só temos uma tarefa, mas outras poderiam ser adicionadas
tarefa_print


ModuleNotFoundError: No module named 'airflow'

In [30]:
import psycopg2
import pandas as pd

# Configurações do banco de dados
db_config = {
    "dbname": "Test",
    "user": "avnadmin",
    "password": "AVNS_9fZb3BkX9qGXxKpxsrZ",
    "host": "postgresql-iscac.f.aivencloud.com",  # ou o IP do servidor
    "port": 25674,  # Porta padrão do PostgreSQL
}

# Caminho do arquivo CSV
csv_file_path = "test.csv"

# Nome da tabela no banco de dados
table_name = "tabela_test"

try:
    # Conectando ao banco de dados
    conn = psycopg2.connect(**db_config)
    cursor = conn.cursor()

    # Lendo o arquivo CSV com pandas
    df = pd.read_csv(csv_file_path)
    df = df.where(pd.notnull(df), None)
    print(df)

    # Iterando pelas linhas do DataFrame para inserção
    for _, row in df.iterrows():
        placeholders = ", ".join(["%s"] * len(row))
        columns = ", ".join(df.columns)
        query = f"INSERT INTO {table_name} ({columns}) VALUES ({placeholders})"
        cursor.execute(query, tuple(row))

    # Confirmando as operações
    conn.commit()
    print(f"Dados inseridos com sucesso na tabela {table_name}.")

except Exception as e:
    print(f"Erro: {e}")
    conn.rollback()

finally:
    if cursor:
        cursor.close()
    if conn:
        conn.close()


   Id    Name  Age      City
0   1    John   30  New York
1   2    Jane   25    London
2   3  Rafael   28      None
Dados inseridos com sucesso na tabela tabela_test.
